In [9]:
import requests
import pandas as pd
from datetime import datetime, date, timedelta
import numpy as np
import re
import math

In [10]:
# гугл монтер, надо будет вводить AUTH код для этого
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
# датасет где-то потерялся, поэтому пришлось перепарсить используя 3ю лабу
df = pd.read_csv('/content/gdrive/My Drive/ABD_Lab_dataset/hh_parsed.csv', delimiter=',')
df = df.drop(columns=['Unnamed: 0'])
df

,name,area,minSalary,maxSalary,employer,published_at,schedule,responsibility,requirement
0,Системный архитектор НИОКР (вычислительная тех...,Москва,120000.0,NaN,Воентелеком,2020-10-11T23:32:08+0300,Полный день,"Разработка концепций, технических требований, ...",Высшее техническое образование. Опыт практичес...
1,Java Developer Trainee,Санкт-Петербург,NaN,NaN,Andersen,2020-10-11T09:32:04+0300,Полный день,NaN,"Уверенное знание принципов ООП, алгоритмов и с..."
2,Java Developer Trainee,Казань,NaN,NaN,Andersen,2020-10-11T09:32:04+0300,Полный день,NaN,"Уверенное знание принципов ООП, алгоритмов и с..."
3,Начальник отдела развития,Волгодонск,NaN,NaN,Пятёрочка,2020-10-11T15:32:07+0300,Полный день,Контроль и реализация выполнения целевых показ...,Высшее финансовое / техническое / юридическое ...
4,Главный инженер контроля качества тестирования,Москва,NaN,NaN,Центральный банк Российской Федерации,2020-10-11T19:32:07+0300,Полный день,Анализ существующих процессов тестирования ПО....,Опыт работы в области IT (QA/Разработка/Тестир...
...,...,...,...,...,...,...,...,...,...
48040,Монтажник ЛКС,Туринск,20000.0,NaN,"К Телеком,ООО",2020-11-10T08:31:15+0300,Полный день,"прокладка слаботочных сетей(интернет), подключ...",опыт в сфере телекоммуникационных услуг и энер...
48041,Программист JS,Ярославль,75000.0,120000.0,Тензор,2020-11-10T08:31:29+0300,Гибкий график,Разработке функционала по учету и контролю раб...,"Хорошее знание JavaScript, html, css. Опыт раб..."
48042,Программист Python,Ярославль,61000.0,90000.0,Тензор,2020-11-10T08:31:38+0300,Гибкий график,Разработке и поддержке существующего функциона...,Знание Python или иных скриптовых языков с пос...
48043,Оператор ЭВМ (ПК),Ульяновск,30000.0,35000.0,Альфа Люкс,2020-11-10T08:33:32+0300,Полный день,Ведение требуемого действующим Российским зако...,"Отличное владение ПК, знание офисных программ ..."


In [12]:
# все уникальные типы занятости (нужный для одного из заданий)
types = df['schedule'].value_counts().keys() 
types

Index(['Полный день', 'Удаленная работа', 'Гибкий график', 'Сменный график',
       'Вахтовый метод'],
      dtype='object')

In [13]:
unique_names = df['name'].value_counts().keys() 
unique_names

Index(['Системный администратор', 'Программист 1С', 'Менеджер по продажам',
       'Оператор саll-центра', 'Специалист технической поддержки',
       'Системный аналитик',
       'Специалист по продаже и установке спутникового ТВ', 'Программист 1C',
       'Frontend-разработчик', 'Менеджер по продажам В2В для Яндекс.Директа',
       ...
       'Руководитель проектов по ИТ-инфраструктуре',
       'Инженер-программист (цифровая обработка сигналов)',
       'Стажер отдела маркетинга',
       'Консультант 1С ( Тверская область, г. Конаково)',
       'Ведущий бизнес-аналитик (ecommerce)',
       'Наставник аналитиков в IT-проекте', 'Product marketing analyst',
       'Ведущий аналитик (МТС ТВ)', 'Менеджер (ИТ-Партнер)',
       'Специалист слаботочных систем'],
      dtype='object', length=24159)



---


Задание: придумать подход и сгруппировать вакансии по названиям (например, Web-разработчик и Web-Developer можно объединить в одну группу);

Исходя из блока кода выше `unique_names` содержит в себе всевозможные варианты
того какие профессии есть (их 24159 штук). 

Для группировки можно использовать несколько подходов:
1.   Группировать по "базовым" словам. Например: `программист`, `администратор` и т.д.
2.   Так как названия могут и не содержать "базовых" слов из пункта 1. То можно сплитать по словам и на их основе делать группы. Проблема данного алгоритма в том, что он может оказаться достаточно турднозатратным по времени + может вызвать небольшую сложность в его написании и дебаге.
3.   Подрубить обученную нейронку и она сама все сделает (такой нейронки не нашел)

Вследствие этого будем использовать подход **1**



---



In [14]:
# норм работает
#df.name.str.lower().str.find('программист') == 0
#df.loc[df.name.str.lower().str.find('программист') == 0, 'name'] = 'программист'
#df.loc[df.name == 'программист']

# массив выделенных мной "базовых слов"
based_words = ['Developer','Разработчик' ,'Программист', 'Специалист', 'Инженер', 'Оператор', 'Администратор', 'Аналитик']
for word in based_words:
  if word.lower() == 'developer':
    df.loc[df.name.str.lower().str.find(word.lower()) == 0, 'category'] = 'Разработчик'
  else:
    df.loc[df.name.str.lower().str.find(word.lower()) == 0, 'category'] = word
# то что не подошло просто ставим другое
df.category.fillna('Другое', inplace=True)
df

,name,area,minSalary,maxSalary,employer,published_at,schedule,responsibility,requirement,category
0,Системный архитектор НИОКР (вычислительная тех...,Москва,120000.0,NaN,Воентелеком,2020-10-11T23:32:08+0300,Полный день,"Разработка концепций, технических требований, ...",Высшее техническое образование. Опыт практичес...,Другое
1,Java Developer Trainee,Санкт-Петербург,NaN,NaN,Andersen,2020-10-11T09:32:04+0300,Полный день,NaN,"Уверенное знание принципов ООП, алгоритмов и с...",Другое
2,Java Developer Trainee,Казань,NaN,NaN,Andersen,2020-10-11T09:32:04+0300,Полный день,NaN,"Уверенное знание принципов ООП, алгоритмов и с...",Другое
3,Начальник отдела развития,Волгодонск,NaN,NaN,Пятёрочка,2020-10-11T15:32:07+0300,Полный день,Контроль и реализация выполнения целевых показ...,Высшее финансовое / техническое / юридическое ...,Другое
4,Главный инженер контроля качества тестирования,Москва,NaN,NaN,Центральный банк Российской Федерации,2020-10-11T19:32:07+0300,Полный день,Анализ существующих процессов тестирования ПО....,Опыт работы в области IT (QA/Разработка/Тестир...,Другое
...,...,...,...,...,...,...,...,...,...,...
48040,Монтажник ЛКС,Туринск,20000.0,NaN,"К Телеком,ООО",2020-11-10T08:31:15+0300,Полный день,"прокладка слаботочных сетей(интернет), подключ...",опыт в сфере телекоммуникационных услуг и энер...,Другое
48041,Программист JS,Ярославль,75000.0,120000.0,Тензор,2020-11-10T08:31:29+0300,Гибкий график,Разработке функционала по учету и контролю раб...,"Хорошее знание JavaScript, html, css. Опыт раб...",Программист
48042,Программист Python,Ярославль,61000.0,90000.0,Тензор,2020-11-10T08:31:38+0300,Гибкий график,Разработке и поддержке существующего функциона...,Знание Python или иных скриптовых языков с пос...,Программист
48043,Оператор ЭВМ (ПК),Ульяновск,30000.0,35000.0,Альфа Люкс,2020-11-10T08:33:32+0300,Полный день,Ведение требуемого действующим Российским зако...,"Отличное владение ПК, знание офисных программ ...",Оператор




---

Задание: добавить новый признак в датасет на основе признака “дата размещения вакансии” - количество дней с момента размещения;

---



In [15]:
# Если делать так то будет OutOfMemory для VM, почему так происходит понять я не могу
# поэтому придется использовать циклы, так как и на подобных манипуляциях происходит out of memory

#days_series = [round(abs(datetime.now() - datetime.strptime(x[:10], "%Y-%m-%d")).days) for x in df.published_at]
#ser = pd.Series(days_series)
#df = pd.concat([df, ser.to_frame().T])
#df

In [16]:
for a in range(len(df)):
  if pd.isna(df.loc[a, 'minSalary']):
    sorted = df.loc[(df['area'] == df.loc[a, 'area']) & (df['category'] == df.loc[a, 'category'])]
    mean = sorted['minSalary'].mean()
    # средняя з/п тоже может быть NaN
    if not math.isnan(mean): 
      df.loc[a, 'minSalary'] = round(mean)
    
  if pd.isna(df.loc[a, 'maxSalary']):
    sorted = df.loc[(df['area'] == df.loc[a, 'area']) & (df['category'] == df.loc[a, 'category'])]
    mean = sorted['maxSalary'].mean()
    if not math.isnan(mean):
      df.loc[a, 'maxSalary'] = round(mean)

  # - добавить новый признак в датасет на основе признака “дата размещения вакансии” - количество дней с момента размещения;
  df.loc[a, 'days'] = round(abs(datetime.now()-datetime.strptime(df.loc[a, 'published_at'][:10], "%Y-%m-%d")).days)
  
  # - заполнить пропуски в признаке “требуемый опыт работы” по принципу: если не указан, то опыт не требуется (нет в датасете);
  
  # - заполнить пропуски в признаке “тип занятости” по принципу: если не указан, то любой тип;
  if pd.isna(df.loc[a, 'schedule']):
    df.loc[a, 'schedule'] = random.choice(types)

  # - при наличии пропусков в остальных признаках заполнить их по методу на свое усмотрение.
  if pd.isna(df.loc[a, 'responsibility']):
    df.loc[a, 'responsibility'] = 'Отсутствуют'
  if pd.isna(df.loc[a, 'requirement']):
    df.loc[a, 'requirement'] = 'Отсутствуют'

df

,name,area,minSalary,maxSalary,employer,published_at,schedule,responsibility,requirement,category,days
0,Системный архитектор НИОКР (вычислительная тех...,Москва,120000.0,143308.0,Воентелеком,2020-10-11T23:32:08+0300,Полный день,"Разработка концепций, технических требований, ...",Высшее техническое образование. Опыт практичес...,Другое,30.0
1,Java Developer Trainee,Санкт-Петербург,86685.0,121062.0,Andersen,2020-10-11T09:32:04+0300,Полный день,Отсутствуют,"Уверенное знание принципов ООП, алгоритмов и с...",Другое,30.0
2,Java Developer Trainee,Казань,57367.0,91784.0,Andersen,2020-10-11T09:32:04+0300,Полный день,Отсутствуют,"Уверенное знание принципов ООП, алгоритмов и с...",Другое,30.0
3,Начальник отдела развития,Волгодонск,29600.0,59500.0,Пятёрочка,2020-10-11T15:32:07+0300,Полный день,Контроль и реализация выполнения целевых показ...,Высшее финансовое / техническое / юридическое ...,Другое,30.0
4,Главный инженер контроля качества тестирования,Москва,98585.0,143308.0,Центральный банк Российской Федерации,2020-10-11T19:32:07+0300,Полный день,Анализ существующих процессов тестирования ПО....,Опыт работы в области IT (QA/Разработка/Тестир...,Другое,30.0
...,...,...,...,...,...,...,...,...,...,...,...
48040,Монтажник ЛКС,Туринск,20000.0,NaN,"К Телеком,ООО",2020-11-10T08:31:15+0300,Полный день,"прокладка слаботочных сетей(интернет), подключ...",опыт в сфере телекоммуникационных услуг и энер...,Другое,0.0
48041,Программист JS,Ярославль,75000.0,120000.0,Тензор,2020-11-10T08:31:29+0300,Гибкий график,Разработке функционала по учету и контролю раб...,"Хорошее знание JavaScript, html, css. Опыт раб...",Программист,0.0
48042,Программист Python,Ярославль,61000.0,90000.0,Тензор,2020-11-10T08:31:38+0300,Гибкий график,Разработке и поддержке существующего функциона...,Знание Python или иных скриптовых языков с пос...,Программист,0.0
48043,Оператор ЭВМ (ПК),Ульяновск,30000.0,35000.0,Альфа Люкс,2020-11-10T08:33:32+0300,Полный день,Ведение требуемого действующим Российским зако...,"Отличное владение ПК, знание офисных программ ...",Оператор,0.0
